# Draw a Sample of 500

In [1]:
import os
import random
import csv
import shutil
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub

from tensorflow import keras

In [2]:
pics = os.listdir(r"/Users/wendyma/Desktop/product image")

len(pics) 

5475

In [3]:
# Draw a sample of 600
random.seed(117)
spl = random.sample(pics, 600)
spl[:5]

['SPRING_AA0402_JUNO_SHORT_3098-ROSE_2737.jpg',
 'CAPERJKTCAMO11.jpg',
 'nova1_bde967e6-114c-428b-a8e9-ec0481da6d7f.jpg',
 'Ashlyn_Black_Web_4.jpg',
 'ANATOMIE_07-12-2019_546.jpg']

In [4]:
# Create a new folder that contains the sampled pics
for i in spl:
    shutil.copyfile(f'/Users/wendyma/Desktop/product image/{i}',f'/Users/wendyma/Desktop/sample_pics/{i}')


In [5]:
# function to Load Images

from PIL import Image
from skimage import transform
def load0(filename):
    np_image = Image.open(filename)
    np_image = np.array(np_image).astype('float32')/255
    np_image = transform.resize(np_image, (224, 224, 3))
    np_image = np.expand_dims(np_image, axis=0)
    return np_image


def load(filename):
    input_image = tf.keras.preprocessing.image.load_img(filename, target_size=(224, 224))
    input_image = tf.keras.preprocessing.image.img_to_array(input_image)
    input_image = np.expand_dims(input_image, axis=0)
    return input_image

In [6]:
# Check readability of images, exclude unreadable images
for i in os.listdir('/Users/wendyma/Desktop/sample_pics/'):
    try:
        image = load(f'/Users/wendyma/Desktop/sample_pics/{i}')
    except:
        os.remove(f'/Users/wendyma/Desktop/sample_pics/{i}')

In [7]:
smp_pics = os.listdir('/Users/wendyma/Desktop/sample_pics/')
print(len(smp_pics), "images are readable and now in the sample_pics file.")

595 images are readable and now in the sample_pics file.


In [8]:
# Leave a sample of 500 of readable images
smp_rm = random.sample(smp_pics, len(smp_pics)-500)
for i in smp_rm:
    os.remove(f'/Users/wendyma/Desktop/sample_pics/{i}')

print("Sample Size: ", len(os.listdir('/Users/wendyma/Desktop/sample_pics')))

Sample Size:  500


In [9]:
#!pip install tensorflow_hub

# Load Models

In [10]:
# Load the trained model
neck_model = keras.models.load_model(r'/Users/wendyma/Documents/MSBA/practicum/image tagging/neck_model_64_folder',
       custom_objects={'KerasLayer':hub.KerasLayer})

# print_model = keras.models.load_model(r"C:\ChromeDownloads\print_model_GCC",
#        custom_objects={'KerasLayer':hub.KerasLayer})

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [11]:
neck_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1001)              25615849  
                                                                 
 dropout (Dropout)           (None, 1001)              0         
                                                                 
 flatten (Flatten)           (None, 1001)              0         
                                                                 
 dense (Dense)               (None, 6)                 6012      
                                                                 
Total params: 25,621,861
Trainable params: 25,576,421
Non-trainable params: 45,440
_________________________________________________________________


In [12]:
# neck_model.summary()

# Apply Models to the Sample data - get the predicted labels

In [13]:
neck_labels = ['Cowl Neck', 'Invisible', 'Round Neck', 'Square Neck', 'Turtle Neck', 'V Neck']

In [14]:
print_labels = ['Animal', 'Camo', 'Floral', 'Other', 'Paisley', 'Plaid', 'Polka', 'Solid', 'Stripe', 'Tiedie']

In [15]:
t = os.listdir("/Users/wendyma/Desktop/sample_pics")[25:29]
for i in t:
    image = load(f'/Users/wendyma/Desktop/sample_pics/{i}')
    neck_outcome = neck_model.predict(image)
    neck_label = neck_labels[np.argmax(neck_outcome)]
    # print_outcome = print_model.predict(image)
    # print_label = print_labels[np.argmax(print_outcome)]
    #Image.open(f'sample_pics/{i}').show()
    #print(neck_outcome)
    print(neck_label)
    #print(print_outcome)
    # print(print_label)

2023-05-02 19:02:14.700314: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


1/1 [==============================] - 0s 326ms/step
Invisible
1/1 [==============================] - 0s 54ms/step
Round Neck
1/1 [==============================] - 0s 54ms/step
Invisible
1/1 [==============================] - 0s 55ms/step
Round Neck


In [16]:
# Predict labels and write labels into a csv file
with open('/Users/wendyma/Desktop/outputs.csv', 'w', newline='') as file:
    csvwriter = csv.writer(file)
    #csvwriter.writerow(['Image', 'Neck', 'Print'])
    csvwriter.writerow(['Image', 'Print', 'Print Prob'])
    for i in os.listdir("/Users/wendyma/Desktop/sample_pics"):
        image = load(f'/Users/wendyma/Desktop/sample_pics/{i}')
        neck_outcome = neck_model.predict(image)
        neck_label = neck_labels[np.argmax(neck_outcome)]
        neck_prob = np.max(neck_outcome)
        # print_outcome = print_model.predict(image)
        # print_label = print_labels[np.argmax(print_outcome)]
        # print_prob = np.max(print_outcome)
        csvwriter.writerow([i, neck_label, neck_prob])
        # csvwriter.writerow([i, print_label, print_prob])

1/1 [==============================] - 0s 50ms/step


In [17]:
pd.read_csv('/Users/wendyma/Desktop/outputs.csv')

,Image,Print,Print Prob
0,Stormi_Emi_Isla_Amrita_1693.jpg,Turtle Neck,0.614019
1,AndiSangria_PrintedStaceySpades_KenyaSangria_M...,Turtle Neck,0.948698
2,sade_green_pdp.jpg,Turtle Neck,0.892828
3,Maria_Ivory_WEB_2.jpg,Invisible,0.695494
4,mandy_pant_black4.jpg,Round Neck,0.955608
...,...,...,...
495,KateSuvaArmyGreen_RonnieCamel_042.jpg,Turtle Neck,0.604001
496,20LOOK_LUISAKHAKI_BARCELONABLK_DELILAHBLK_0432...,Turtle Neck,0.651527
497,Steph_Web_3.jpg,Invisible,0.673565
498,ANATOMIE_10-27-2018_1091.jpg,V Neck,0.999969


# Manually Check the Correctness of the 500 Outcomes

In [22]:
import os
import csv
import pandas as pd
from PIL import Image

In [23]:
# Write the checked outputs
with open('/Users/wendyma/Desktop/outputs.csv','r') as file:
    lines = file.readlines()[1:]
    with open('/Users/wendyma/Desktop/checked_output.csv','w') as csvoutput:
        writer = csv.writer(csvoutput)
        writer.writerow(['Image', 'Print', 'Print Prob', 'Print_Check'])
        num = 0
        for line in lines:
            line = line.split(",")
            print(num, "Print: ",line[1])
            Image.open(f"/Users/wendyma/Desktop/sample_pics/{line[0]}").show()
            check = input()
            if check == 'quit':
                print("Next time start with: ",num)
                break
            else:
                writer.writerow(line+[check])
                num += 1

0 Print:  Turtle Neck
1 Print:  Turtle Neck
2 Print:  Turtle Neck
3 Print:  Invisible
4 Print:  Round Neck
5 Print:  Invisible
6 Print:  Square Neck
7 Print:  Turtle Neck
8 Print:  Invisible
9 Print:  Invisible
10 Print:  V Neck
11 Print:  V Neck
12 Print:  Square Neck
13 Print:  Invisible
14 Print:  Turtle Neck
15 Print:  Round Neck
16 Print:  Turtle Neck
17 Print:  V Neck
18 Print:  Invisible
19 Print:  Square Neck
20 Print:  Cowl Neck
21 Print:  Invisible
22 Print:  V Neck
23 Print:  Invisible
24 Print:  Square Neck
25 Print:  Invisible
26 Print:  Round Neck
27 Print:  Invisible
28 Print:  Round Neck
29 Print:  V Neck
30 Print:  Invisible
31 Print:  Invisible
32 Print:  Invisible
33 Print:  Invisible
34 Print:  Invisible
35 Print:  Round Neck
36 Print:  V Neck
37 Print:  Invisible
38 Print:  Invisible
39 Print:  Invisible
40 Print:  Invisible
41 Print:  V Neck
42 Print:  Invisible
43 Print:  Invisible
44 Print:  Turtle Neck
45 Print:  Turtle Neck
46 Print:  V Neck
47 Print:  V Neck


In [24]:
with open('/Users/wendyma/Desktop/outputs.csv','r') as file:
    lines = file.readlines()[num:70]
    for line in lines:
        print(line.split(","))

In [28]:
# Continue to work
num = 400

with open('/Users/wendyma/Desktop/outputs.csv','r') as file:
    lines = file.readlines()[num:]
    with open('/Users/wendyma/Desktop/checked_output.csv','a') as csvoutput:
        writer = csv.writer(csvoutput)
        for line in lines:
            line = line.split(',')
            print(num, "Print: ",line[1])
            Image.open(f"/Users/wendyma/Desktop/sample_pics/{line[0]}").show()
            check = input()
            if check == 'quit':
                print("Next time start with: ",num)
                break
            else:
                writer.writerow(line+[check])
                num += 1

400 Print:  Turtle Neck
401 Print:  V Neck
402 Print:  Invisible
403 Print:  Invisible
404 Print:  Invisible
405 Print:  V Neck
406 Print:  Invisible
407 Print:  Turtle Neck
408 Print:  Turtle Neck
409 Print:  Invisible
410 Print:  Turtle Neck
411 Print:  Square Neck
412 Print:  Round Neck
413 Print:  Turtle Neck
414 Print:  V Neck
415 Print:  Invisible
416 Print:  Invisible
417 Print:  V Neck
418 Print:  Invisible
419 Print:  Turtle Neck
Next time start with:  419
